### Author: Akshay Ijantkar
### Team: Aqua Wizards
### Project: Surfers Bible

* https://launchschool.com/books/sql/read/table_relationships

# CRONTAB EXPRESSIONA and STATEMENT:

0 2 * * * /usr/bin/python3 /home/ubuntu/pop_db_sch_ss/Daily_Scheduler_Weather_Tide_API.py >> /home/ubuntu/pop_db_sch_ss/log_Daily_Scheduler_Weather_Tide_API.txt 2>&1

In [ ]:
# Traceback (most recent call last):
#   File "/home/ubuntu/pop_db_sch_ss/Daily_Scheduler_Weather_Tide_API.py", line 1377, in <module>
#     model.load_model('catboost_model_rand_search_tide_weather_shark_feat')
#   File "/home/ubuntu/.local/lib/python3.6/site-packages/catboost/core.py", line 2589, in load_model
#     self._load_model(fname, format)
#   File "/home/ubuntu/.local/lib/python3.6/site-packages/catboost/core.py", line 1315, in _load_model
#     self._object._load_model(model_file, format)
#   File "_catboost.pyx", line 4658, in _catboost._CatBoost._load_model
#   File "_catboost.pyx", line 4661, in _catboost._CatBoost._load_model
# _catboost.CatBoostError: catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: catboost_model_rand_search_tide_weather_shark_feat


In [ ]:
# log_Daily_Scheduler_Weather_Tide_API

# Import Libraries:

In [3]:
import pandas as pd
import numpy as np
import random
import matplotlib as plt
from matplotlib import pyplot
# import seaborn as sns; sns.set()
# from scipy.stats import norm 
import matplotlib.pyplot as plt
# For Linear regression
# from sklearn.linear_model import LinearRegression
# For split given dataset into train and test set.
# from sklearn.model_selection import train_test_split
# To verify models using this metrics 
# from sklearn.metrics import mean_squared_error, r2_score

# import statsmodels.formula.api as smf
# from sklearn.linear_model import LinearRegression
from sklearn import metrics
# v
from matplotlib import rcParams
# rcParams['figure.figsize'] = 50,50
# import pandas_profiling
# pd.set_option('display.max_rows', 1500)
# pd.set_option('display.max_columns', 500)
# pd.set_option('display.width', 1000)
from pandas import ExcelWriter
from pandas import ExcelFile

# from pygeocoder import Geocoder

import sys
# from weather_au import api
# from weather_au import summary
# from weather import place, observations, uv_index
import time

import json
import requests
from datetime import datetime, timedelta
from catboost import CatBoostClassifier
# from sklearn.model_selection import train_test_split

# from sklearn.metrics import r2_score
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import KFold
# from sklearn.model_selection import cross_validate

import catboost as ctb
# from catboost import CatBoostRegressor, FeaturesData, Pool
# from scipy.stats import uniform as sp_randFloat
# from scipy.stats import randint as sp_randInt
# from scipy.stats import uniform
# from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics.pairwise import cosine_similarity
# from  sklearn.metrics.pairwise import euclidean_distances
# from sklearn.metrics.pairwise import manhattan_distances
# from sklearn.metrics.pairwise import pairwise_distances
import re
import pprint
from datetime import date
import datetime
# import sqlite3
# from sqlite3 import Error
from datetime import datetime
from dateutil import tz

# %matplotlib inline

# Decide Date:

In [6]:
from datetime import datetime
import datetime
no_days_from_today = 3

select_date = "2020-05-18"

if select_date == "":    
    today = date.today()
    today_date = today.strftime("%Y-%m-%d") 
    given_date =  str((datetime.datetime.strptime(today_date, "%Y-%m-%d") + datetime.timedelta(days = no_days_from_today)).date())
    given_date_timestamp_epoch = int(time.mktime(time.strptime(given_date, '%Y-%m-%d')))
    print("today_date = ", today_date)
    print("given_date = ", given_date)
    print("given_date_timestamp_epoch = ",given_date_timestamp_epoch)
else:
    given_date = select_date
    given_date_timestamp_epoch = int(time.mktime(time.strptime(given_date, '%Y-%m-%d')))
    
    print("given_date = ", given_date)
    print("given_date_timestamp_epoch = ",given_date_timestamp_epoch)


given_date =  2020-05-18
given_date_timestamp_epoch =  1589724000


In [7]:
# date_5days_lst = []
# for day in range(5):
#     date_5days_lst.append(str((datetime.datetime.strptime(given_date, "%Y-%m-%d") + datetime.timedelta(days = day)).date()))
# date_5days_lst    

In [8]:
# timestamp_epochs_5days_lst = []
# for date in date_5days_lst:
# #     + 3600
#     timestamp_epochs_5days_lst.append(int(time.mktime(time.strptime(date, '%Y-%m-%d'))) )
# timestamp_epochs_5days_lst    

# READ BEACH DATABASE FROM RDS POSTGRES DB:

In [9]:
import psycopg2 as ps

# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

# Import Beach Table to DF: 

In [10]:
beach_df = pd.read_sql_query("SELECT * FROM BEACH_TABLE;", conn)
# beach_df.head()

In [11]:
# beach_df.columns.tolist()

# Weather API:

In [12]:
weather_beach_req_col_lst = ['date',
 'beach_id',
 'beach_name',
 'beach_latitude',
 'beach_longitude',
 'beach_state',
 'time',
 'summary',
 'icon',
 'precipIntensity',
 'precipProbability',
 'temperature',
 'apparentTemperature',
 'dewPoint',
 'humidity',
 'pressure',
 'windSpeed',
 'windGust',
 'windBearing',
 'cloudCover',
 'uvIndex',
 'visibility',
 'ozone']

weather_df = pd.DataFrame(columns = weather_beach_req_col_lst
                         )
# weather_df  

In [13]:
# %%time
# Wall time: 24min 24s
API_KEY = "bdbb908566b674b2e5971ac7f61e1bfc"
cnt = 0
# https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-33.869,151.209,2019-12-30T12:00:00

# for date in date_5days_lst[:]:
    
#     timestamp_epochs = int(time.mktime(time.strptime(date, '%Y-%m-%d')))

for index, row in beach_df.loc[:,].iterrows():

    print("beach_name = ", row["beach_name"])
    print("date = ",given_date,"\n")
    print("index = ", index)

    get_request = ""
    get_request += "https://api.darksky.net/forecast/"
    get_request += API_KEY
    get_request += "/"
    get_request += str(row["beach_latitude"])
    get_request += ","
    get_request += str(row["beach_longitude"])
    get_request += ","
    get_request += str(given_date_timestamp_epoch)

    response_dict = json.loads(requests.get(get_request).text)

    print("get_request = ", get_request)
    print("****************************************************************************************************")
#     print("response_dict = ",response_dict)

    if len(list(response_dict.keys())) > 2:
        cnt = cnt + 1
        print("cnt =>>>>>>>>>>>>>>>>>>>>> ", cnt)

        weather_row_dict = {}

        for per_hr_attri_dict in response_dict['hourly']['data']:

            weather_row_dict["date"] = given_date

            for col in ['beach_id', 'beach_name', 'beach_latitude', 'beach_longitude', 'beach_state']:
                weather_row_dict[col] = row[col]

            for hr_attri in per_hr_attri_dict.keys():
                weather_row_dict[hr_attri] = per_hr_attri_dict[hr_attri]                    

            weather_row_dict["nearest-station"] = response_dict['flags']['nearest-station']
            weather_row_dict["sources"] = response_dict['flags']['sources']
            weather_row_dict["offset"] = response_dict['offset']            

            weather_df = weather_df.append(weather_row_dict, ignore_index=True)


    else:
        print("############################################################################################")
        print("Response Failed...!")
        print("beach_name = ", row["beach_name"])
        print("date = ",date,"\n")
        print("index = ", index)
        print("############################################################################################")
#     time.sleep(1)
# weather_df            

beach_name =  ADDISCOT BEACH
date =  2020-05-18 

index =  0
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.3823,144.257,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  1
beach_name =  APEX BEACH
date =  2020-05-18 

index =  1
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-35.813,145.547,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  2
beach_name =  BACK BEACH
date =  2020-05-18 

index =  2
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.5269,145.37,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  3
beach_name =  BANCOORA BEACH
date =  2020-05-18 

index =  3
get_request =  https://api.darksky.net/forecas

get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-35.8218,145.451,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  28
beach_name =  CHERRY TREE BEACH
date =  2020-05-18 

index =  28
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-37.9856,147.652,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  29
beach_name =  CHILDRENS BEACH
date =  2020-05-18 

index =  29
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.5228,145.364,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  30
beach_name =  CHINAMAN BEACH
date =  2020-05-18 

index =  30
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.83

get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-37.8865,144.981,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  55
beach_name =  FARM BEACH
date =  2020-05-18 

index =  55
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.4893,145.152,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  56
beach_name =  FINLEY BEACH
date =  2020-05-18 

index =  56
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-35.8221,145.559,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  57
beach_name =  FISHERMANS BEACH
date =  2020-05-18 

index =  57
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.2256,145.0

beach_name =  KILLARNEY BEACH
date =  2020-05-18 

index =  82
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.3564,142.31,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  83
beach_name =  KOONYA BEACH
date =  2020-05-18 

index =  83
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.3541,144.764,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  84
beach_name =  KOONYA OCEAN BEACH
date =  2020-05-18 

index =  84
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.3541,144.764,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  85
beach_name =  LABBETT BEACH
date =  2020-05-18 

index =  85
get_request =  https://api.d

beach_name =  MULBERRY BEACH
date =  2020-05-18 

index =  109
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-35.8184,145.559,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  110
beach_name =  MYRONG BEACH
date =  2020-05-18 

index =  110
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-36.1206,143.732,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  111
beach_name =  NETTLE PASS
date =  2020-05-18 

index =  111
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.7539,143.322,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  112
beach_name =  NEVINS BEACH EAST
date =  2020-05-18 

index =  112
get_request =  https://a

beach_name =  REEVES BEACH
date =  2020-05-18 

index =  136
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.5752,146.954,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  137
beach_name =  RICARDO BEACH
date =  2020-05-18 

index =  137
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-37.8061,148.62,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  138
beach_name =  RIPPLESIDE BEACH
date =  2020-05-18 

index =  138
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.1291,144.357,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  139
beach_name =  RIVERNOOK BEACH
date =  2020-05-18 

index =  139
get_request =  https://

beach_name =  SOUTHCOMBE BEACH
date =  2020-05-18 

index =  163
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.3922,142.231,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  164
beach_name =  SQUEAKY BEACH
date =  2020-05-18 

index =  164
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-39.0253,146.305,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  165
beach_name =  ST ANDREWS BEACH
date =  2020-05-18 

index =  165
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.4271,144.835,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  166
beach_name =  ST HELENS BEACH
date =  2020-05-18 

index =  166
get_request =  htt

beach_name =  WILD DOG SHORE
date =  2020-05-18 

index =  190
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-38.1328,147.339,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  191
beach_name =  WILLOW BEACH
date =  2020-05-18 

index =  191
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-35.8488,145.584,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  192
beach_name =  WILSONS BEACH
date =  2020-05-18 

index =  192
get_request =  https://api.darksky.net/forecast/bdbb908566b674b2e5971ac7f61e1bfc/-35.8688,145.602,1589724000
****************************************************************************************************
cnt =>>>>>>>>>>>>>>>>>>>>>  193
beach_name =  WOODSIDE BEACH
date =  2020-05-18 

index =  193
get_request =  https://ap

# Convert timestamp epochs to datetime format:

In [14]:
weather_df["datetime"] = weather_df["time"].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x)))

* https://launchschool.com/books/sql/read/table_relationships

In [15]:
import psycopg2 as ps

# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

create_table_query = '''
      CREATE TABLE IF NOT EXISTS WEATHER_TABLE
      (
        weather_id SERIAL PRIMARY KEY,
        date DATE,
        beach_id INTEGER NOT NULL,
        beach_name TEXT,
        beach_latitude REAL,
        beach_longitude REAL,
        beach_state TEXT,
        time REAL,
        summary TEXT,
        icon TEXT,
        precipIntensity REAL,
        precipProbability REAL,
        temperature REAL,
        apparentTemperature REAL,
        dewPoint REAL,
        humidity REAL,
        pressure REAL,
        windSpeed REAL,
        windGust REAL,
        windBearing REAL,
        cloudCover REAL,
        uvIndex REAL,
        visibility REAL,
        ozone REAL,
        nearest_station REAL,
        time_offset REAL,
        precipType TEXT,
        sources TEXT,
        datetime TEXT,
        FOREIGN KEY (beach_id) REFERENCES beach_table(beach_id) ON DELETE CASCADE
       ); 
       '''

cur.execute(create_table_query)
conn.commit()

cur.close()
conn.close()


In [16]:
# %%time
# Wall time: 12min 16s
import psycopg2 as ps

# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

fill_question_mark_str = str(tuple(["%s"  for i in weather_df.columns.tolist()])).replace("'", "")
fill_question_mark_str

for row in weather_df.itertuples():
    data_tuple = tuple(row[1:])

#     print("data_tuple = ", data_tuple)
#     print(" ")
    
    cur.execute("""
                        INSERT INTO WEATHER_TABLE
                        (
                            date,
                            beach_id,
                            beach_name,
                            beach_latitude,
                            beach_longitude,
                            beach_state,
                            time,
                            summary,
                            icon,
                            precipIntensity,
                            precipProbability,
                            temperature,
                            apparentTemperature,
                            dewPoint,
                            humidity,
                            pressure,
                            windSpeed,
                            windGust,
                            windBearing,
                            cloudCover,
                            uvIndex,
                            visibility,
                            ozone,
                            nearest_station,
                            time_offset,
                            precipType,
                            sources,
                            datetime
                         ) VALUES  
                         """ + fill_question_mark_str + " ;"
                , data_tuple)    

conn.commit()

cur.close()
conn.close()

In [17]:
weather_df["date_beach_name"] = weather_df[
                                                ['date', 'beach_name']
                                             ].apply(lambda x: '|'.join(x.astype(str).values), axis=1)
# weather_df.head()

In [18]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# weather_df.to_csv(log_dataset_path + "weather_df"+"_10_05"+".csv")

## AGGREGATE WEATHER:

In [19]:
agg_ops_lst = ['max','min','mean','median','std','var','sem']
weather_numeric_features = ["temperature", "apparentTemperature", "dewPoint", "humidity", "windSpeed", "windBearing", "uvIndex",
                    "cloudCover"]

agg_dict = dict(zip(weather_numeric_features, [agg_ops_lst for i in range(len(weather_numeric_features))]))
# agg_dict

In [20]:
for col in weather_numeric_features:
    weather_df[col] = weather_df[col].astype(float) 

In [21]:
weather_beach_req_col_lst = [
    "date_beach_name",
    'date',
     'beach_name',
     'beach_latitude',
     'beach_longitude',
     'beach_state',
]

agg_weather_df = weather_df[weather_numeric_features + ["date_beach_name"]].groupby('date_beach_name').agg(agg_dict)

agg_weather_df.columns = ["_".join(x) for x in agg_weather_df.columns.ravel()]
agg_weather_df.reset_index(level=0, inplace=True)
agg_weather_df =  pd.merge(
                           agg_weather_df, 
                           weather_df[weather_beach_req_col_lst],
                           left_on = "date_beach_name",
                           right_on = "date_beach_name",
                           how = "inner"
                            )

agg_weather_df.drop_duplicates(inplace = True)
agg_weather_df.reset_index(inplace = True, 
                           drop = True)

print("agg_weather_df.shape = ", agg_weather_df.shape)
# agg_weather_df.head()

agg_weather_df.shape =  (200, 62)


In [22]:
# agg_weather_df.isnull().sum()

In [23]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# agg_weather_df.to_csv(log_dataset_path + "agg_weather_df"+"_10_05"+".csv")

# END OF WEATHER API PROCESSING:

---
---
---

# TIDE API PROCESSING STARTS:

In [24]:
heights_tide_df = pd.DataFrame(columns = [
                                         'date',
                                        'beach_id',
                                        'beach_name',
                                        'beach_latitude',
                                        'beach_longitude',
                                        'beach_state',
                                         'timestamp', 
                                         'datetime', 
                                         'height', 
                                         'state', 
                                         'origin_distance',
                                         "origin_distance_unit", 
                                         "origin_latitude", 
                                         "origin_longitude", 
                                         "timezone"
                                        ]
                              )

# heights_tide_df 
extremes_tide_df = pd.DataFrame(columns = [
                                        'date',
                                        'beach_id',
                                        'beach_name',
                                        'beach_latitude',
                                        'beach_longitude',
                                        'beach_state',   
                                        'timestamp', 
                                        'datetime', 
                                        'height', 
                                        'state'
                                          ]
                               )

# extremes_tide_df
datum_tide_df = pd.DataFrame(columns = [
                                        'date',
                                        'beach_id',
                                        'beach_name',
                                        'beach_latitude',
                                        'beach_longitude',
                                        'beach_state',                
                                        'timestamp', 
                                        'datetime', 
                                        'datum', 
                                        'LAT', 
                                        'HAT'
                                        ]
                            )

In [25]:
# %%time
# Wall time: 14min 4s
# datum_tide_df
cnt = 0
for index, row in beach_df.loc[:,:].iterrows():
#     if index >= 0:
    print("beach_name = ", row["beach_name"])
    print("index = ", index)
    timestamp_epochs = int(time.mktime(time.strptime(given_date, '%Y-%m-%d')))

    url = "https://tides.p.rapidapi.com/tides"    
    querystring = {"interval":"60",
                   "duration":"1440",
#                    "duration": "7200",
                   "timestamp": str(timestamp_epochs),
                   "latitude":str(row["beach_latitude"]),
                   "longitude":str(row["beach_longitude"])}

    headers = {
        'x-rapidapi-host': "tides.p.rapidapi.com",
        'x-rapidapi-key': "1f79b4df37msh714afcfc6dcaf4ep10f0dfjsn67fa86f634ec",
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }

    response_dict = json.loads(requests.request("GET", url, headers = headers, params = querystring).text)
#     print("response_dict - ", response_dict)
    print(" ")
    if response_dict['status'] == 200:
        cnt = cnt + 1
        print("cnt = ", cnt)

#         Updating heights_tide_df 
#                                          'date',
#                                         'beach_id',
#                                         'beach_name',
#                                         'beach_latitude',
#                                         'beach_longitude',
#                                         'beach_state',  
#                                          'timestamp', 
#                                          'datetime', 
#                                          'height', 
#                                          'state', 
#                                          'origin_distance',
#                                          "origin_distance_unit", 
#                                          "origin_latitude", 
#                                          "origin_longitude", 
#                                          "timezone"

        heights_tide_row_dict = {}
        heights_tide_row_dict["date"] = given_date
        heights_tide_row_dict["beach_id"] = row["beach_id"]
        heights_tide_row_dict["beach_name"] = row["beach_name"]
        heights_tide_row_dict["beach_latitude"] = row["beach_latitude"]
        heights_tide_row_dict["beach_longitude"] = row["beach_longitude"]
        heights_tide_row_dict["beach_state"] = row["beach_state"]
        

        heights_tide_row_dict["origin_distance"] = response_dict['origin']['distance']
        heights_tide_row_dict["origin_distance_unit"] = response_dict['origin']['unit']
        heights_tide_row_dict["origin_latitude"] = response_dict['origin']['latitude']
        heights_tide_row_dict["origin_longitude"] = response_dict['origin']['longitude']
        heights_tide_row_dict["timezone"] = response_dict["timezone"]

        for ht_dict in response_dict['heights']:
            heights_tide_row_dict["timestamp"] = ht_dict['timestamp']
            heights_tide_row_dict["datetime"] = ht_dict['datetime']
            heights_tide_row_dict["height"] = ht_dict['height']
            heights_tide_row_dict["state"] = ht_dict['state']

            heights_tide_df = heights_tide_df.append(heights_tide_row_dict, 
                                                 ignore_index = True)

#         Updating extremes_tide_df:
#                                         'date',
#                                         'beach_address',
#                                         'beach_name',
#                                         'country_state',
#                                         'country',
#                                         'latitude',
#                                         'longitude',    
#                                         'timestamp', 
#                                         'datetime', 
#                                         'height', 
#                                         'state'

        extremes_tide_row_dict = {}
        extremes_tide_row_dict["date"] = given_date
        extremes_tide_row_dict["beach_id"] = row["beach_id"]
        extremes_tide_row_dict["beach_name"] = row["beach_name"]
        extremes_tide_row_dict["beach_latitude"] = row["beach_latitude"]
        extremes_tide_row_dict["beach_longitude"] = row["beach_longitude"]
        extremes_tide_row_dict["beach_state"] = row["beach_state"] 

        for extremes_dict in response_dict['extremes']:

            extremes_tide_row_dict["timestamp"] = extremes_dict['timestamp']
            extremes_tide_row_dict["datetime"] = extremes_dict['datetime']
            extremes_tide_row_dict["height"] = extremes_dict['height']
            extremes_tide_row_dict["state"] = extremes_dict['state']   

            extremes_tide_df = extremes_tide_df.append(extremes_tide_row_dict, 
                                                 ignore_index = True)

        #         Updating datum_tide_df
#                                         'date',
#                                         'beach_address',
#                                         'beach_name',
#                                         'country_state',
#                                         'country',
#                                         'latitude',
#                                         'longitude',                
#                                         'timestamp', 
#                                         'datetime', 
#                                         'datum', 
#                                         'LAT', 
#                                         'HAT'
        datum_tide_row_dict = {}
        datum_tide_row_dict["date"] = given_date
        datum_tide_row_dict["beach_id"] = row["beach_id"]
        datum_tide_row_dict["beach_name"] = row["beach_name"]
        datum_tide_row_dict["beach_latitude"] = row["beach_latitude"]
        datum_tide_row_dict["beach_longitude"] = row["beach_longitude"]
        datum_tide_row_dict["beach_state"] = row["beach_state"] 

        datum_tide_row_dict["timestamp"] = response_dict["timestamp"]
        datum_tide_row_dict["datetime"] = response_dict["datetime"]
        datum_tide_row_dict["datum"] = response_dict["datum"]
        datum_tide_row_dict["LAT"] = response_dict['datums']["LAT"]
        datum_tide_row_dict["HAT"] = response_dict['datums']["HAT"]

        datum_tide_df = datum_tide_df.append(
                                            datum_tide_row_dict,
                                            ignore_index = True
                                            )
        print(" ")
        time.sleep(1)

beach_name =  ADDISCOT BEACH
index =  0
 
cnt =  1
 
beach_name =  APEX BEACH
index =  1
 
cnt =  2
 
beach_name =  BACK BEACH
index =  2
 
cnt =  3
 
beach_name =  BANCOORA BEACH
index =  3
 
cnt =  4
 
beach_name =  BARRY BEACH
index =  4
 
cnt =  5
 
beach_name =  BARRYS BEACH
index =  5
 
cnt =  6
 
beach_name =  BAY BEACH
index =  6
 
cnt =  7
 
beach_name =  BELL BEACH
index =  7
 
cnt =  8
 
beach_name =  BELLS BEACH
index =  8
 
cnt =  9
 
beach_name =  BERRY BEACH
index =  9
 
cnt =  10
 
beach_name =  BETKA BEACH
index =  10
 
cnt =  11
 
beach_name =  BIDDLES BEACH
index =  11
 
cnt =  12
 
beach_name =  BIRDROCK BEACH
index =  12
 
cnt =  13
 
beach_name =  BIRRELLS BEACH
index =  13
 
cnt =  14
 
beach_name =  BLACK BEACH
index =  14
 
cnt =  15
 
beach_name =  BOOMANGONG BEACH
index =  15
 
cnt =  16
 
beach_name =  BOUCHIERS BEACH
index =  16
 
cnt =  17
 
beach_name =  BOURKES BEACH NO. 1
index =  17
 
cnt =  18
 
beach_name =  BOURKES BEACH NO. 2
index =  18
 
cnt =  1

 
cnt =  147
 
beach_name =  SALMON BEACH
index =  147
 
cnt =  148
 
beach_name =  SANDRIDGE BEACH
index =  148
 
cnt =  149
 
beach_name =  SANDY WATERHOLE BEACH
index =  149
 
cnt =  150
 
beach_name =  SCOTTS BEACH
index =  150
 
cnt =  151
 
beach_name =  SCOUT BEACH
index =  151
 
cnt =  152
 
beach_name =  SECRET BEACH
index =  152
 
cnt =  153
 
beach_name =  SHELLEY BEACH
index =  153
 
cnt =  154
 
beach_name =  SHELLY BEACH
index =  154
 
cnt =  155
 
beach_name =  SHIRE HALL BEACH
index =  155
 
cnt =  156
 
beach_name =  SHOREHAM BEACH
index =  156
 
cnt =  157
 
beach_name =  SMITH BEACH
index =  157
 
cnt =  158
 
beach_name =  SMITHS BEACH
index =  158
 
cnt =  159
 
beach_name =  SOMERS BEACH
index =  159
 
cnt =  160
 
beach_name =  SORRENTO BACK BEACH
index =  160
 
cnt =  161
 
beach_name =  SORRENTO FRONT BEACH
index =  161
 
cnt =  162
 
beach_name =  SOUTH BEACH
index =  162
 
cnt =  163
 
beach_name =  SOUTHCOMBE BEACH
index =  163
 
cnt =  164
 
beach_name =  S

In [26]:
print("heights_tide_df.shape = ",heights_tide_df.shape)
# heights_tide_df.head()

heights_tide_df.shape =  (4800, 15)


In [27]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# heights_tide_df.to_csv(log_dataset_path + "heights_tide_df"+"_10_05"+".csv")

In [28]:
print("extremes_tide_df.shape = ", extremes_tide_df.shape)
# extremes_tide_df.head()

extremes_tide_df.shape =  (820, 10)


In [29]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# extremes_tide_df.to_csv(log_dataset_path + "extremes_tide_df"+"_10_05"+".csv")

## CONVERT UTC TIME to LOCAL TIME:

In [30]:
def convert_datetime_in_dif_timezones(from_datetime_str, from_timezone_str, to_timezone_str, 
                                      datetime_format = '%Y-%m-%dT%H:%M:%S'):
#     USE pytz.all_timezones to get all timestamps
    from datetime import datetime
    import pytz
    date_time_obj = datetime.strptime(from_datetime_str, datetime_format)
#     print("date_time_obj = ", date_time_obj)
    
    old_timezone = pytz.timezone(from_timezone_str)
    new_timezone = pytz.timezone(to_timezone_str)
    
    new_timezone_timestamp = old_timezone.localize(date_time_obj).astimezone(new_timezone).strftime("%Y-%m-%dT%H:%M:%S") 
#     print("new_timezone_timestamp", new_timezone_timestamp)
    return str(new_timezone_timestamp)
    

In [31]:
heights_tide_df["datetime"] = heights_tide_df["datetime"].apply(lambda x: convert_datetime_in_dif_timezones(from_datetime_str = x, 
                                                                                  from_timezone_str = 'UTC', 
                                                                                  to_timezone_str ='Australia/Melbourne', 
                                                                                datetime_format = '%Y-%m-%dT%H:%M:%S+00:00'))

extremes_tide_df["datetime"] = extremes_tide_df["datetime"].apply(lambda x: convert_datetime_in_dif_timezones(from_datetime_str = x, 
                                                                                  from_timezone_str = 'UTC', 
                                                                                  to_timezone_str ='Australia/Melbourne', 
                                                                                datetime_format = '%Y-%m-%dT%H:%M:%S+00:00'))

# Get date from Date time:

In [32]:
heights_tide_df['date'] = heights_tide_df['datetime'].apply(lambda x: x.split("T")[0])
extremes_tide_df['date'] = extremes_tide_df['datetime'].apply(lambda x: x.split("T")[0])

## Create date_coordinates column which will be used as PK:

In [33]:
heights_tide_df["date_beach_name"] = heights_tide_df[
                                                     ['date', 'beach_name']
                                                     ].apply(lambda x: '|'.join(x.astype(str).values), axis=1)
# heights_tide_df.head()

In [34]:
extremes_tide_df["date_beach_name"] = extremes_tide_df[
                                                     ['date', 'beach_name']
                                                     ].apply(lambda x: '|'.join(x.astype(str).values), axis=1)
# extremes_tide_df.head()

# CREATE TIDE_HEIGHT_24HR_TABLE:

In [35]:
# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

create_table_query = """ 
                            CREATE TABLE IF NOT EXISTS TIDE_HEIGHT_TABLE (
                                tide_height_id SERIAL PRIMARY KEY ,
                                date DATE,
                                beach_id INTEGER NOT NULL,
                                beach_name TEXT,
                                beach_latitude REAL,
                                beach_longitude REAL,
                                beach_state TEXT,
                                timestamp INTEGER,
                                datetime TEXT,
                                height REAL,
                                state TEXT,
                                origin_distance REAL,
                                origin_distance_unit TEXT,
                                origin_latitude REAL,
                                origin_longitude REAL,
                                timezone TEXT,
                                date_beach_name TEXT,
                                FOREIGN KEY (beach_id) REFERENCES beach_table(beach_id) ON DELETE CASCADE
                                ); 
                            """

cur.execute(create_table_query)
conn.commit()

cur.close()
conn.close()

# INSERTION TIDE_HEIGHT_24HR_TABLE:

In [36]:
# %%time
# Wall time: 12min 51s
import psycopg2 as ps

# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

fill_question_mark_str = str(tuple(["%s"  for i in heights_tide_df.columns.tolist()])).replace("'", "")
fill_question_mark_str

for row in heights_tide_df.itertuples():
    data_tuple = tuple(row[1:])

#     print("data_tuple = ", data_tuple)
#     print(" ")
    
    cur.execute("""
                        INSERT INTO TIDE_HEIGHT_TABLE
                        (
                                date,
                                beach_id,
                                beach_name,
                                beach_latitude,
                                beach_longitude,
                                beach_state,
                                timestamp,
                                datetime,
                                height,
                                state,
                                origin_distance,
                                origin_distance_unit,
                                origin_latitude,
                                origin_longitude,
                                timezone,
                                date_beach_name
                         ) VALUES  
                         """ + fill_question_mark_str + " ;"
                , data_tuple)    

conn.commit()
cur.close()
conn.close()

# CREATE EXTREMES_HEIGHT_24HR_TABLE:

In [37]:
# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

create_table_query = """ 
                            CREATE TABLE IF NOT EXISTS EXTREMES_HEIGHT_TABLE (
                                extremes_height_id SERIAL PRIMARY KEY ,
                                date DATE,
                                beach_id INTEGER NOT NULL,
                                beach_name TEXT,
                                beach_latitude REAL,
                                beach_longitude REAL,
                                beach_state TEXT,
                                timestamp INTEGER,
                                datetime TEXT,
                                height REAL,
                                state TEXT,
                                date_beach_name TEXT,
                                FOREIGN KEY (beach_id) REFERENCES beach_table(beach_id) ON DELETE CASCADE
                                ); 
                            """

cur.execute(create_table_query)
conn.commit()
cur.close()
conn.close()

# INSERTION EXTREMES_HEIGHT_24HR_TABLE:

In [38]:
# %%time
# Wall time: 12min 51s
import psycopg2 as ps

# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

fill_question_mark_str = str(tuple(["%s"  for i in extremes_tide_df.columns.tolist()])).replace("'", "")
fill_question_mark_str

for row in extremes_tide_df.itertuples():
    data_tuple = tuple(row[1:])

#     print("data_tuple = ", data_tuple)
#     print(" ")
    
    cur.execute("""
                        INSERT INTO EXTREMES_HEIGHT_TABLE
                        (
                            date,
                            beach_id,
                            beach_name,
                            beach_latitude,
                            beach_longitude,
                            beach_state,
                            timestamp,
                            datetime,
                            height,
                            state,
                            date_beach_name
                         ) VALUES  
                         """ + fill_question_mark_str + " ;"
                , data_tuple)    

conn.commit()
cur.close()
conn.close()

# AGGREGATES TIDES:

In [39]:
req_beach_loc_attri_lst = [ 
                            "date",
                            "beach_id",
                            "beach_name",
                            "beach_latitude",
                            "beach_longitude",
                            "beach_state",
                            "date_beach_name",
                            ]

## HEIGHTS FALL TIDES AGG:

In [40]:
agg_ops_lst = ['max','min','mean','median','std','var','sem']
numeric_features = ["height"]
agg_dict = dict(zip(numeric_features, [agg_ops_lst for i in range(len(numeric_features))]))
# agg_dict

agg_fall_heights_tide_df = heights_tide_df.loc[heights_tide_df['state'] == "FALLING",:
                                              ].groupby([
                                                        'date_beach_name',
                                                        ]).agg(agg_dict)


agg_fall_heights_tide_df.columns = ["_fall_".join(x) for x in agg_fall_heights_tide_df.columns.ravel()]

agg_fall_heights_tide_df =  pd.merge(
                               agg_fall_heights_tide_df, 
                               heights_tide_df[req_beach_loc_attri_lst],
                               left_on = "date_beach_name",
                               right_on = "date_beach_name",
                               how = "inner"
                                    )

agg_fall_heights_tide_df.drop_duplicates(inplace = True)

agg_fall_heights_tide_df.reset_index(inplace = True, 
                                     drop = True)

print("agg_fall_heights_tide_df.shape = ", agg_fall_heights_tide_df.shape)
# agg_fall_heights_tide_df.head()

agg_fall_heights_tide_df.shape =  (200, 14)


In [41]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# agg_fall_heights_tide_df.to_csv(log_dataset_path + "agg_fall_heights_tide_df"+"_10_05"+".csv")

## HEIGHTS RISE TIDES AGG:

In [42]:
agg_ops_lst = ['max','min','mean','median','std','var','sem']
numeric_features = ["height"]
agg_dict = dict(zip(numeric_features, [agg_ops_lst for i in range(len(numeric_features))]))
# agg_dict

agg_rise_heights_tide_df = heights_tide_df.loc[heights_tide_df['state'] == "RISING",:
                                              ].groupby([
                                                        'date_beach_name',
                                                        ]
                                                       ).agg(agg_dict)

agg_rise_heights_tide_df.columns = ["_rise_".join(x) for x in agg_rise_heights_tide_df.columns.ravel()]

agg_rise_heights_tide_df =  pd.merge(
                               agg_rise_heights_tide_df, 
                               heights_tide_df[req_beach_loc_attri_lst],
                               left_on = "date_beach_name",
                               right_on = "date_beach_name",
                               how = "inner"
                                    )

agg_rise_heights_tide_df.drop_duplicates(inplace = True)

agg_rise_heights_tide_df.reset_index(inplace = True, 
                                     drop = True)

print("agg_rise_heights_tide_df.shape = ", agg_rise_heights_tide_df.shape)
# agg_rise_heights_tide_df.head()

agg_rise_heights_tide_df.shape =  (200, 14)


In [43]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# agg_rise_heights_tide_df.to_csv(log_dataset_path + "agg_rise_heights_tide_df"+"_10_05"+".csv")

## EXTREMES HIGH TIDES AGG:

In [44]:
agg_ops_lst = ['max','min','mean']
numeric_features = ["height"]
agg_dict = dict(zip(numeric_features, [agg_ops_lst for i in range(len(numeric_features))]))
# agg_dict

agg_high_tide_extremes_tide_df = extremes_tide_df.loc[extremes_tide_df['state'] == "HIGH TIDE",:
                                              ].groupby([
                                                        'date_beach_name',
                                                        ]).agg(agg_dict)

agg_high_tide_extremes_tide_df.columns = ["_high_tide_".join(x) for x in agg_high_tide_extremes_tide_df.columns.ravel()]

agg_high_tide_extremes_tide_df =  pd.merge(
                               agg_high_tide_extremes_tide_df, 
                               extremes_tide_df[req_beach_loc_attri_lst],
                               left_on = "date_beach_name",
                               right_on = "date_beach_name",
                               how = "inner"
                                    )

agg_high_tide_extremes_tide_df.drop_duplicates(inplace = True)

agg_high_tide_extremes_tide_df.reset_index(inplace = True, 
                                     drop = True)

print("agg_high_tide_extremes_tide_df.shape = ", agg_high_tide_extremes_tide_df.shape)
# agg_high_tide_extremes_tide_df.head()

agg_high_tide_extremes_tide_df.shape =  (200, 10)


In [45]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# agg_high_tide_extremes_tide_df.to_csv(log_dataset_path + "agg_high_tide_extremes_tide_df"+"_10_05"+".csv")

## EXTREMES LOW TIDES AGG:

In [46]:
agg_ops_lst = ['max','min','mean']
numeric_features = ["height"]
agg_dict = dict(zip(numeric_features, [agg_ops_lst for i in range(len(numeric_features))]))
# agg_dict

agg_low_tide_extremes_tide_df = extremes_tide_df.loc[extremes_tide_df['state'] == "LOW TIDE",:
                                              ].groupby(['date_beach_name']).agg(agg_dict)

agg_low_tide_extremes_tide_df.columns = ["_low_tide_".join(x) for x in agg_low_tide_extremes_tide_df.columns.ravel()]

agg_low_tide_extremes_tide_df =  pd.merge(
                               agg_low_tide_extremes_tide_df, 
                               extremes_tide_df[req_beach_loc_attri_lst],
                               left_on = "date_beach_name",
                               right_on = "date_beach_name",
                               how = "inner"
                                    )

agg_low_tide_extremes_tide_df.drop_duplicates(inplace = True)

agg_low_tide_extremes_tide_df.reset_index(inplace = True, 
                                     drop = True)

print("agg_low_tide_extremes_tide_df.shape = ", agg_low_tide_extremes_tide_df.shape)
# agg_low_tide_extremes_tide_df.head()

agg_low_tide_extremes_tide_df.shape =  (200, 10)


In [47]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# agg_low_tide_extremes_tide_df.to_csv(log_dataset_path + "agg_low_tide_extremes_tide_df"+"_10_05"+".csv")

# END OF TIDE API PROCESSING:

---
---
---

# Joing WEATHER AND TIDE AGG DF:

In [48]:
# agg_weather_df.head()

In [49]:
# agg_rise_heights_tide_df.head()

In [50]:
tide_df_lst = [
                agg_rise_heights_tide_df, agg_fall_heights_tide_df,
                agg_high_tide_extremes_tide_df, agg_low_tide_extremes_tide_df,
#                 agg_datum_tide_df,
              ]
join_weather_tide_df = agg_weather_df.copy()
for df in tide_df_lst:
    join_weather_tide_df = pd.merge(
                               join_weather_tide_df, 
                               df.drop(['date', 'beach_name', 'beach_name', 'beach_state', 
                                        'beach_latitude','beach_longitude'], axis = 1, inplace = False),
                               left_on = "date_beach_name",
                               right_on = "date_beach_name",
                               how = "inner")
# drop_col_lst = []
# for col in join_weather_tide_df.columns.tolist():
#     if col.endswith("_x")|col.endswith("_y"):
#         drop_col_lst.append(col)
        
# join_weather_tide_df.drop(drop_col_lst, 
#                           axis=1, 
#                           inplace=True)

        
# join_weather_tide_df.head()

In [51]:
# join_weather_tide_df.shape

In [52]:
def get_month_season_month_day_func(date):
    month = date.split("-")[1]
    month_day = date.split("-")[2]
    if (int(month) == 9) | (int(month) == 10) | (int(month) == 11):
        season = "spring"
    elif (int(month) == 12) | (int(month) == 1) | (int(month) == 2):
        season = "summer"
    elif (int(month) == 3) | (int(month) == 4) | (int(month) == 5):
        season = "autumn"
    elif (int(month) == 6) | (int(month) == 7) | (int(month) == 8):
        season = "winter"
    return month, season, month_day
join_weather_tide_df['month'], join_weather_tide_df['season'], join_weather_tide_df['month_day'] = zip(*join_weather_tide_df.apply(lambda x: 
                                                                                        get_month_season_month_day_func(
                                                                                            x['date']), 
                                                                                        axis = 1))
# join_weather_tide_df.head()

# ML MODEL INFERENCE:

In [53]:
numeric_features_lst = [
#     Using Weather features
    'temperature_max',
 'temperature_min',
 'temperature_mean',
 'temperature_median',
 'temperature_std',
 'temperature_var',
 'temperature_sem',
 'apparentTemperature_max',
 'apparentTemperature_min',
 'apparentTemperature_mean',
 'apparentTemperature_median',
 'apparentTemperature_std',
 'apparentTemperature_var',
 'apparentTemperature_sem',
 'dewPoint_max',
 'dewPoint_min',
 'dewPoint_mean',
 'dewPoint_median',
 'dewPoint_std',
 'dewPoint_var',
 'dewPoint_sem',
 'humidity_max',
 'humidity_min',
 'humidity_mean',
 'humidity_median',
 'humidity_std',
 'humidity_var',
 'humidity_sem',
 'windSpeed_max',
 'windSpeed_min',
 'windSpeed_mean',
 'windSpeed_median',
 'windSpeed_std',
 'windSpeed_var',
 'windSpeed_sem',
 'windBearing_max',
 'windBearing_min',
 'windBearing_mean',
 'windBearing_median',
 'windBearing_std',
 'windBearing_var',
 'windBearing_sem',
 'uvIndex_max',
 'uvIndex_min',
 'uvIndex_mean',
 'uvIndex_median',
 'uvIndex_std',
 'uvIndex_var',
 'uvIndex_sem',
 'cloudCover_max',
 'cloudCover_min',
 'cloudCover_mean',
 'cloudCover_median',
 'cloudCover_std',
 'cloudCover_var',
 'cloudCover_sem',
# Tides Features                        
 'height_fall_max',
 'height_fall_min',
 'height_fall_mean',
 'height_fall_median',
 'height_fall_std',
 'height_fall_var',
 'height_fall_sem',
 'height_rise_max',
 'height_rise_min',
 'height_rise_mean',
 'height_rise_median',
 'height_rise_std',
 'height_rise_var',
 'height_rise_sem',
 'height_high_tide_max',
 'height_high_tide_min',
 'height_high_tide_mean',
 'height_low_tide_max',
 'height_low_tide_min',
 'height_low_tide_mean',
#  'LAT_datum_mean',
#  'HAT_datum_mean',
 'beach_latitude',
 'beach_longitude'
]
cat_features_lst = [
 'month',
 'month_day',
 'season']

# SHARK ATTACK PREDICTION:

In [54]:
params = {'loss_function':'Logloss', # objective function
          'eval_metric':'Accuracy', # metric
          'verbose': 200, # output to stdout info about training process every 200 iterations
          'random_seed': 100
         }

model = CatBoostClassifier(**params)
model.load_model('catboost_model_rand_search_tide_weather_shark_feat')

In [55]:
join_weather_tide_df["shark_attack_percentage"] = 0.0
for index, row in join_weather_tide_df[numeric_features_lst+cat_features_lst].iterrows():
#     print(model.predict_proba(row.values)[1])
    join_weather_tide_df.loc[index, "shark_attack_percentage"] = round(float(model.predict_proba(row.values)[1])*100.0, 3)
#     join_weather_tide_df.loc[index, "shark_attack_percentage"] = model.predict_proba(row.values)[1]

# join_weather_tide_df.head(5)

# SHARK SIGHTING PREDICTION:

# Load CSV clean addresses with Coordinates CSV as Dataframe:

In [58]:
dataset_path = r"D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\\"
# dataset_path = r""
join_shark_weather_tide_df = pd.read_csv(dataset_path+"for_ml_join_shark_weather_tide_df.csv")
# raw_shark_df = pd.read_excel(dataset_path + "shark_file_geolocation_checkpoint.xlsx")
print("join_shark_weather_tide_df.shape = ", join_shark_weather_tide_df.shape)
join_shark_weather_tide_df.rename(columns = {
                                            "lat":"beach_latitude",
                                            "lng":"beach_longitude"}, inplace = True)
# join_shark_weather_tide_df.tail(5)

join_shark_weather_tide_df.shape =  (707, 102)


# Using Cosine Similarity: Comparing point with distribution

In [59]:
# %%time
# Wall time: 2min 36s
threshold = 0.90
join_weather_tide_df['shark_sighting_percentage'] = 0.0
for index_o, row_o in join_weather_tide_df[numeric_features_lst].iterrows():
    cos_sim_lst = []
    for index_i, row_i in join_shark_weather_tide_df[numeric_features_lst].iterrows():
    
        cosine_similarity_score = cosine_similarity(
                                                    X = row_o.values.reshape(1, -1), 
                                                    Y = row_i.values.reshape(1, -1), 
                                                    dense_output=True).flatten()[0]
        
        cos_sim_lst.append(cosine_similarity_score)
    
    no_vals_above = len([i for i in cos_sim_lst if i > threshold])
    
    percent_vals_above = round((no_vals_above/float(join_shark_weather_tide_df.shape[0]))*100.0, 3)
    
    join_weather_tide_df.loc[index_o, 'shark_sighting_percentage'] = percent_vals_above

In [60]:
def percent_2_level_func(percent):
    if (percent > 0) & (percent <= 25):
        return "Low"
    elif (percent > 25) & (percent <= 50):
        return "Moderately Low"
    elif (percent > 50) & (percent <= 75):
        return "Moderately High"
    elif (percent > 75) & (percent <= 100):
        return "High"

In [61]:
join_weather_tide_df["shark_sighting_level"] = join_weather_tide_df["shark_sighting_percentage"].apply(percent_2_level_func)
join_weather_tide_df["shark_attack_level"] = join_weather_tide_df["shark_attack_percentage"].apply(percent_2_level_func)
# join_weather_tide_df.head(10)

In [62]:
# log_dataset_path = "D:\Monash_University_Stuff\Final_Semester\IE\Surfers_Bible_Code_Commit\Datasets\Daily_weather_tide_log\\"
# join_weather_tide_df.to_csv(log_dataset_path + "join_weather_tide_df"+"_10_05"+".csv")

In [63]:
# join_weather_tide_df
join_weather_tide_df = join_weather_tide_df.loc[:,~join_weather_tide_df.columns.duplicated()]
join_weather_tide_df.drop(['beach_id_y'], axis=1, inplace=True)
join_weather_tide_df.rename(columns = {"beach_id_x": "beach_id"}, inplace =True)

# CREATE join_weather_tide_TABLE:

In [64]:
# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

create_table_query = """ 
                            CREATE TABLE IF NOT EXISTS SHARK_PREDICTION_TABLE (
                                shark_prediction_id SERIAL PRIMARY KEY ,
                                date_beach_name TEXT,
                                temperature_max REAL,
                                temperature_min REAL,
                                temperature_mean REAL,
                                temperature_median REAL,
                                temperature_std REAL,
                                temperature_var REAL,
                                temperature_sem REAL,
                                apparentTemperature_max REAL,
                                apparentTemperature_min REAL,
                                apparentTemperature_mean REAL,
                                apparentTemperature_median REAL,
                                apparentTemperature_std REAL,
                                apparentTemperature_var REAL,
                                apparentTemperature_sem REAL,
                                dewPoint_max REAL,
                                dewPoint_min REAL,
                                dewPoint_mean REAL,
                                dewPoint_median REAL,
                                dewPoint_std REAL,
                                dewPoint_var REAL,
                                dewPoint_sem REAL,
                                humidity_max REAL,
                                humidity_min REAL,
                                humidity_mean REAL,
                                humidity_median REAL,
                                humidity_std REAL,
                                humidity_var REAL,
                                humidity_sem REAL,
                                windSpeed_max REAL,
                                windSpeed_min REAL,
                                windSpeed_mean REAL,
                                windSpeed_median REAL,
                                windSpeed_std REAL, 
                                windSpeed_var REAL,
                                windSpeed_sem REAL,
                                windBearing_max REAL,
                                windBearing_min REAL,
                                windBearing_mean REAL,
                                windBearing_median REAL,
                                windBearing_std REAL,
                                windBearing_var REAL,
                                windBearing_sem REAL,
                                uvIndex_max REAL,
                                uvIndex_min REAL,
                                uvIndex_mean REAL,
                                uvIndex_median REAL,
                                uvIndex_std REAL,
                                uvIndex_var REAL,
                                uvIndex_sem REAL,
                                cloudCover_max REAL,
                                cloudCover_min REAL,
                                cloudCover_mean REAL,
                                cloudCover_median REAL,
                                cloudCover_std REAL,
                                cloudCover_var REAL,
                                cloudCover_sem REAL,
                                date DATE,
                                beach_name TEXT,
                                beach_latitude REAL,
                                beach_longitude REAL,
                                beach_state TEXT,
                                height_rise_max REAL,
                                height_rise_min REAL,
                                height_rise_mean REAL,
                                height_rise_median REAL,
                                height_rise_std REAL,
                                height_rise_var REAL,
                                height_rise_sem REAL,
                                beach_id INTEGER,
                                height_fall_max REAL,
                                height_fall_min REAL,
                                height_fall_mean REAL,
                                height_fall_median REAL,
                                height_fall_std REAL,
                                height_fall_var REAL,
                                height_fall_sem REAL,
                                height_high_tide_max REAL,
                                height_high_tide_min REAL,
                                height_high_tide_mean REAL,
                                height_low_tide_max REAL,
                                height_low_tide_min REAL,
                                height_low_tide_mean REAL,
                                month TEXT,
                                season TEXT,
                                month_day TEXT,
                                shark_attack_percentage REAL,
                                shark_sighting_percentage REAL,
                                shark_sighting_level TEXT,
                                shark_attack_level TEXT,
                                FOREIGN KEY (beach_id) REFERENCES beach_table(beach_id) ON DELETE CASCADE
                                ); 
                            """

cur.execute(create_table_query)
conn.commit()

cur.close()
conn.close()

# INSERTION join_weather_tide_TABLE:

In [65]:
# %%time
# Wall time: 34 s
import psycopg2 as ps

# define credentials 
credentials = {'POSTGRES_ADDRESS' : 'test-surfers-bible-instance.cljoljhkgpfb.ap-southeast-2.rds.amazonaws.com', # change to your endpoint
               'POSTGRES_PORT' : 5432, # change to your port
               'POSTGRES_USERNAME' : 'ai_postgres', # change to your username
               'POSTGRES_PASSWORD' : 'postgres2309', # change to your password
               'POSTGRES_DBNAME' : 'test_surfers_bible_db'} # change to your db name

# create connection and cursor    
conn = ps.connect(host=credentials['POSTGRES_ADDRESS'],
                  database=credentials['POSTGRES_DBNAME'],
                  user=credentials['POSTGRES_USERNAME'],
                  password=credentials['POSTGRES_PASSWORD'],
                  port=credentials['POSTGRES_PORT'])

cur = conn.cursor()

fill_question_mark_str = str(tuple(["%s"  for i in join_weather_tide_df.columns.tolist()])).replace("'", "")
fill_question_mark_str

for row in join_weather_tide_df.itertuples():
    data_tuple = tuple(row[1:])

    print("data_tuple = ", data_tuple)
    print(" ")
    
    cur.execute("""
                        INSERT INTO SHARK_PREDICTION_TABLE
                        (
                            date_beach_name,
                            temperature_max,
                            temperature_min,
                            temperature_mean,
                            temperature_median,
                            temperature_std,
                            temperature_var,
                            temperature_sem,
                            apparentTemperature_max,
                            apparentTemperature_min,
                            apparentTemperature_mean,
                            apparentTemperature_median,
                            apparentTemperature_std,
                            apparentTemperature_var,
                            apparentTemperature_sem,
                            dewPoint_max,
                            dewPoint_min,
                            dewPoint_mean,
                            dewPoint_median,
                            dewPoint_std,
                            dewPoint_var,
                            dewPoint_sem,
                            humidity_max,
                            humidity_min,
                            humidity_mean,
                            humidity_median,
                            humidity_std,
                            humidity_var,
                            humidity_sem,
                            windSpeed_max,
                            windSpeed_min,
                            windSpeed_mean,
                            windSpeed_median,
                            windSpeed_std,
                            windSpeed_var,
                            windSpeed_sem,
                            windBearing_max,
                            windBearing_min,
                            windBearing_mean,
                            windBearing_median,
                            windBearing_std,
                            windBearing_var,
                            windBearing_sem,
                            uvIndex_max,
                            uvIndex_min,
                            uvIndex_mean,
                            uvIndex_median,
                            uvIndex_std,
                            uvIndex_var,
                            uvIndex_sem,
                            cloudCover_max,
                            cloudCover_min,
                            cloudCover_mean,
                            cloudCover_median,
                            cloudCover_std,
                            cloudCover_var,
                            cloudCover_sem,
                            date,
                            beach_name,
                            beach_latitude,
                            beach_longitude,
                            beach_state,
                            height_rise_max,
                            height_rise_min,
                            height_rise_mean,
                            height_rise_median,
                            height_rise_std,
                            height_rise_var,
                            height_rise_sem,
                            beach_id,
                            height_fall_max,
                            height_fall_min,
                            height_fall_mean,
                            height_fall_median,
                            height_fall_std,
                            height_fall_var,
                            height_fall_sem,
                            height_high_tide_max,
                            height_high_tide_min,
                            height_high_tide_mean,
                            height_low_tide_max,
                            height_low_tide_min,
                            height_low_tide_mean,
                            month,
                            season,
                            month_day,
                            shark_attack_percentage,
                            shark_sighting_percentage,
                            shark_sighting_level,
                            shark_attack_level
                         ) VALUES  
                         """ + fill_question_mark_str + " ;"
                , data_tuple)    

conn.commit()

cur.close()
conn.close()

data_tuple =  ('2020-05-18|ADDISCOT BEACH', 65.74, 44.1, 53.52583333333333, 54.095, 7.503819269091462, 56.30730362318832, 1.531710694278192, 65.74, 40.93, 52.427083333333336, 54.095, 8.707755902775343, 75.82501286231881, 1.7774632305423232, 48.99, 37.5, 42.893333333333324, 43.260000000000005, 4.113524529885655, 16.921084057971, 0.8396696785534924, 0.79, 0.54, 0.6804166666666668, 0.71, 0.08848872031614206, 0.007830253623188413, 0.018062684397199952, 11.61, 4.39, 7.216666666666666, 7.4, 2.046529759854719, 4.188284057971014, 0.41774613792205467, 359.0, 3.0, 167.25, 15.0, 174.91097114271295, 30593.847826086967, 35.70355247619332, 2.0, 0.0, 0.4583333333333333, 0.0, 0.7790276362049131, 0.6068840579710145, 0.15901835035238, 0.99, 0.01, 0.5454166666666668, 0.645, 0.42796768637329397, 0.18315634057971011, 0.08735853816783598, '2020-05-18', 'ADDISCOT BEACH', -38.3823, 144.257, 'VIC', 0.5275179622864015, -0.3918208136408383, 0.06083636858822381, 0.057084579257108636, 0.299758362051678, 0.08985507

data_tuple =  ('2020-05-18|BAY BEACH', 62.84, 50.4, 56.41041666666666, 57.4, 4.601294221682933, 21.171908514492745, 0.9392352499616543, 62.84, 50.4, 56.41041666666666, 57.4, 4.601294221682933, 21.171908514492745, 0.9392352499616543, 49.28, 42.54, 45.86625000000001, 45.57, 2.829345926104697, 8.005198369565244, 0.577537818731569, 0.75, 0.61, 0.6787499999999999, 0.69, 0.048750696762021845, 0.0023766304347826073, 0.009951194306008802, 16.93, 6.63, 11.606666666666667, 12.079999999999998, 3.655632541074433, 13.363649275362315, 0.7462028677288525, 49.0, 1.0, 15.541666666666666, 13.0, 11.894387301385574, 141.4764492753624, 2.4279316409528713, 2.0, 0.0, 0.4583333333333333, 0.0, 0.7790276362049131, 0.6068840579710145, 0.15901835035238, 1.0, 0.06, 0.5354166666666667, 0.51, 0.4170077589379995, 0.17389547101449274, 0.08512135234830251, '2020-05-18', 'BAY BEACH', -38.342, 144.749, 'VIC', 0.41581034604810835, -0.2694266340147661, 0.06420308786377552, 0.052841977232904166, 0.23874282253346563, 0.05699

data_tuple =  ('2020-05-18|BIRDROCK BEACH', 62.78, 46.76, 54.891249999999985, 56.44, 5.811622023452511, 33.77495054347826, 1.186292377948312, 62.78, 44.01, 54.09499999999999, 56.44, 6.82438275597141, 46.57219999999994, 1.3930212967981972, 51.28, 43.34, 47.82916666666667, 47.79, 3.119744856717365, 9.732807971014452, 0.6368152522191468, 0.88, 0.65, 0.7770833333333332, 0.78, 0.0847235487343685, 0.007178079710144916, 0.017294121966418864, 11.74, 4.79, 8.672916666666667, 10.045, 2.6474507494974273, 7.008995471014489, 0.5404086212847988, 358.0, 0.0, 74.25, 13.0, 129.98469809608528, 16896.021739130436, 26.533015392094086, 2.0, 0.0, 0.4583333333333333, 0.0, 0.7790276362049131, 0.6068840579710145, 0.15901835035238, 1.0, 0.02, 0.5008333333333334, 0.405, 0.40385013004359904, 0.16309492753623187, 0.08243556259695405, '2020-05-18', 'BIRDROCK BEACH', -38.2493, 145.028, 'VIC', 0.42165774530908956, -0.2844557691172482, 0.03235435481250457, 0.0366181384453217, 0.24135792506292875, 0.058253647990682327,

data_tuple =  ('2020-05-18|BURGES BEACH', 69.21, 44.36, 55.85250000000001, 57.3, 8.402245688427332, 70.59773260869568, 1.7151012191789112, 69.21, 42.83, 55.543333333333344, 57.3, 8.797132767415544, 77.38954492753628, 1.7957072066405142, 51.47, 43.5, 47.82416666666666, 48.83, 3.030230177319252, 9.182294927536267, 0.6185431448012967, 0.97, 0.48, 0.7641666666666667, 0.8, 0.1557287682971978, 0.02425144927536232, 0.03178800171668702, 7.62, 2.73, 5.34, 5.6850000000000005, 1.9325315678843555, 3.7346782608695652, 0.3944763544281185, 352.0, 2.0, 136.70833333333334, 30.5, 154.02667165043246, 23724.215579710137, 31.440562693563905, 3.0, 0.0, 0.5416666666666666, 0.0, 0.9315329426211431, 0.8677536231884059, 0.19014836566792676, 0.32, 0.0, 0.12500000000000003, 0.12, 0.09413219197074608, 0.008860869565217392, 0.019214653224836634, '2020-05-18', 'BURGES BEACH', -37.428, 149.714, 'VIC', 0.41891127072841194, -0.4356053522846736, 0.043013440623811974, 0.04605074697140618, 0.2818261256347634, 0.0794259650

data_tuple =  ('2020-05-18|CHINAMAN LONG BEACH', 60.27, 48.87, 54.043749999999996, 54.0, 3.751091218044023, 14.070685326086993, 0.7656882885702411, 60.27, 46.3, 53.64458333333334, 54.0, 4.347246984078509, 18.898556340579688, 0.8873780747371178, 50.94, 45.34, 48.32416666666666, 48.415, 2.0833416231719166, 4.3403123188405965, 0.4252603280560723, 0.88, 0.69, 0.8125, 0.84, 0.06257169800560268, 0.003915217391304342, 0.012772394371104202, 6.02, 2.85, 4.822916666666666, 5.0649999999999995, 0.9670955133201987, 0.9352737318840587, 0.1974075450141216, 89.0, 14.0, 48.166666666666664, 41.0, 20.029688110330163, 401.1884057971015, 4.0885429647833265, 3.0, 0.0, 0.5, 0.0, 0.8846517369293827, 0.7826086956521737, 0.1805787796286538, 1.0, 0.0, 0.3754166666666667, 0.2, 0.3802742622013139, 0.1446085144927536, 0.0776231587255466, '2020-05-18', 'CHINAMAN LONG BEACH', -38.8366, 146.419, 'VIC', 0.5486634239054395, -0.4347105865161695, 0.0669106355417614, 0.0857950769587269, 0.3370962032760249, 0.11363385026311

...

data_tuple =  ('2020-05-18|STATION BEACH', 65.05, 50.4, 57.215833333333336, 58.805, 5.1769538774529655, 26.800851449275292, 1.0567412851402203, 65.05, 50.4, 57.215833333333336, 58.805, 5.1769538774529655, 26.800851449275292, 1.0567412851402203, 50.78, 42.27, 46.529583333333335, 46.215, 3.2506554244345565, 10.566760688405807, 0.6635372599562455, 0.75, 0.57, 0.6779166666666666, 0.7, 0.05808383496342137, 0.003373731884057971, 0.011856313163700966, 13.76, 5.39, 8.222499999999998, 7.4, 2.211334930288274, 4.890002173913046, 0.4513868524666068, 355.0, 0.0, 83.45833333333333, 13.0, 140.70906746427775, 19799.04166666667, 28.72211812252788, 2.0, 0.0, 0.4166666666666667, 0.0, 0.7755316082290384, 0.6014492753623187, 0.15830472663009768, 1.0, 0.06, 0.6150000000000001, 0.7649999999999999, 0.3865679534165962, 0.1494347826086957, 0.07890785306521983, '2020-05-18', 'STATION BEACH', -38.8295, 143.485, 'VIC', 0.2234040076186659, -0.12771349402000617, 0.02585274433114478, 0.019995899425727073, 0.111762075

data_tuple =  ('2020-05-18|THE MAHOGANYS', 62.94, 45.25, 53.888749999999995, 54.86, 5.736961256320434, 32.91272445652173, 1.1710523126751167, 62.94, 43.4, 53.403333333333336, 54.86, 6.325346298591203, 40.01000579710143, 1.2911559064958913, 51.86, 44.72, 48.43375, 48.165, 2.2634464150743705, 5.1231896739130205, 0.4620240647553356, 1.0, 0.62, 0.82875, 0.84, 0.12428272467311738, 0.015446195652173898, 0.025369104940995573, 6.94, 1.68, 3.8804166666666675, 3.7350000000000003, 1.4487970397260068, 2.0990128623188404, 0.29573445734862386, 359.0, 0.0, 134.91666666666666, 91.0, 142.52441946883187, 20313.210144927536, 29.092675298752564, 3.0, 0.0, 0.5416666666666666, 0.0, 0.9315329426211431, 0.8677536231884059, 0.19014836566792676, 0.51, 0.0, 0.16083333333333336, 0.125, 0.16818511195720617, 0.028286231884057975, 0.034330642219001416, '2020-05-18', 'THE MAHOGANYS', -37.7754, 148.98, 'VIC', 0.3918907052984931, -0.3245546701378329, 0.06543941454069784, 0.07662101220989581, 0.2428610048983057, 0.05898

data_tuple =  ('2020-05-18|TIP BEACH', 68.47, 48.15, 57.727916666666665, 59.565, 7.066516871019685, 49.93566068840583, 1.4424467160639165, 68.47, 46.13, 57.340833333333336, 59.565, 7.591557047523063, 57.63173840579708, 1.549620093305091, 51.42, 44.95, 48.705833333333345, 49.865, 2.4909958139120865, 6.205060144927538, 0.5084723912911242, 0.89, 0.52, 0.7320833333333332, 0.73, 0.12311109040094702, 0.015156340579710149, 0.02512994609666436, 11.88, 3.58, 8.119583333333335, 8.57, 3.3123661307376593, 10.971769384057971, 0.6761339051320251, 358.0, 4.0, 126.875, 28.5, 153.5808277493448, 23587.070652173912, 31.349555188347516, 3.0, 0.0, 0.5416666666666666, 0.0, 0.9315329426211431, 0.8677536231884059, 0.19014836566792676, 0.39, 0.0, 0.13416666666666666, 0.11, 0.11706247557781185, 0.013703623188405796, 0.023895277766054705, '2020-05-18', 'TIP BEACH', -37.5739, 149.759, 'VIC', 0.41697673108229744, -0.4329052928086792, 0.04198540545918927, 0.045371155127535874, 0.2803398475808472, 0.0785904301416526

data_tuple =  ('2020-05-18|WOODSIDE BEACH', 61.66, 44.34, 51.70291666666666, 51.275, 5.618962057306424, 31.57273460144924, 1.1469658270383294, 61.66, 41.45, 50.662499999999994, 51.275, 6.718330052598801, 45.1359586956522, 1.3713733793727279, 49.7, 40.81, 44.95291666666668, 45.25, 3.052198750105394, 9.31591721014493, 0.6230274609432334, 0.87, 0.62, 0.78125, 0.805, 0.07531455774746108, 0.00567228260869565, 0.015373519723721439, 5.52, 4.31, 5.013333333333333, 5.1, 0.31718416905868607, 0.10060579710144914, 0.06474494739020475, 357.0, 2.0, 54.041666666666664, 16.0, 96.03825287088866, 9223.346014492752, 19.60372627683824, 3.0, 0.0, 0.5, 0.0, 0.8846517369293827, 0.7826086956521737, 0.1805787796286538, 0.85, 0.04, 0.3066666666666666, 0.25, 0.22928369870698093, 0.05257101449275361, 0.046802339014178206, '2020-05-18', 'WOODSIDE BEACH', -38.5522, 146.978, 'VIC', 0.46507907052954234, -0.4212860999053231, 0.06250993349683266, 0.11035416200869846, 0.284510620284379, 0.08094629305460209, 0.0821311416

data_tuple =  ('2020-05-18|ZINETTIS BEACH NO. 2', 64.47, 42.87, 52.00833333333333, 51.290000000000006, 7.262230862582743, 52.73999710144928, 1.482396667301655, 64.47, 38.82, 50.59916666666667, 51.290000000000006, 8.750961148039261, 76.57932101449263, 1.7862824643013564, 50.19, 41.88, 45.73666666666666, 46.459999999999994, 3.096249718423984, 9.5867623188406, 0.6320193271979044, 0.97, 0.59, 0.8029166666666668, 0.855, 0.12295206441913667, 0.015117210144927532, 0.025097485054057653, 9.02, 4.62, 6.97375, 6.890000000000001, 1.1876458720474312, 1.4105027173913036, 0.2424271984699138, 114.0, 41.0, 69.66666666666667, 68.0, 21.361620738598237, 456.31884057971024, 4.360422574035067, 3.0, 0.0, 0.625, 0.0, 1.0959411282189107, 1.2010869565217388, 0.22370804602220382, 0.69, 0.0, 0.13958333333333334, 0.0, 0.2479389680628238, 0.06147373188405797, 0.05061032992551108, '2020-05-18', 'ZINETTIS BEACH NO. 2', -35.97, 145.936, 'VIC', 0.43630061580150786, -0.27604955464153097, 0.03312805669724651, 0.008324726

---
---
---